# Contour extraction for ground truth masks of DAVIS 2016

In [13]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import pandas as pd
import re
from scipy import spatial
from skimage import measure

%matplotlib inline

## Functions

### Save functions

In [14]:
def save_folder_list_as_csv(annotations_folder_id, file_name):
    '''Saves Drive folder list with title and id as csv.'''
    
    folder_list = drive.ListFile({'q': ("'{}' in parents and trashed=false").
                                  format(ANNOTATIONS_480P_FOLDER_ID)}).GetList()
    df = pd.DataFrame.from_dict(folder_list)
    df = df[['title', 'id']]
    df.to_csv(file_name)

In [15]:
def save_image_with_contour(image, contour, file_name):
    '''Saves plotted image with contour.'''
    
    fig, ax = plt.subplots(figsize=(15, 10))
    ax.scatter(contour[:, 0], contour[:, 1])
    ax.imshow(image)
    ax.axis('image')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.savefig(file_name, dpi=300, bbox_inches='tight')    
    plt.close(fig)

### Contour functions

In [16]:
def close_image(image, kernel_size):
    '''Returns the image that is closed with a kernel.'''
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_size, kernel_size))
    closing = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    
    return closing

In [17]:
def extract_longest_contour(image, kernel_size):
    '''Returns the contour with the most points from a given image.'''
    
    # Close image
    closing = close_image(image, kernel_size)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(closing, cv2.COLOR_BGR2GRAY)

    # Apply threshold to turn it into binary image
    ret, thresh = cv2.threshold(gray, 0, 255, 0)

    # Change method for different number of points:
    # CHAIN_APPROX_NONE, CHAIN_APPROX_SIMPLE, CHAIN_APPROX_TC89_L1, CHAIN_APPROX_TC89_KCOS
    _, contours, _ = cv2.findContours(image=thresh, 
                                      mode=cv2.RETR_TREE,
                                      method=cv2.CHAIN_APPROX_NONE)
    
    # Get longest contour
    longest_contour = max(contours, key=len)
    
    # Remove single-dimensional entry from the shape of the array 
    longest_contour = np.squeeze(longest_contour)
    
    return longest_contour

In [18]:
def shorten_contour(contour, points):
    '''Returns a contour with a fixed number of points for a given contour.'''
    
    # TODO: add warning if short_contour has less than stated points
    try:
        step = len(contour) / (len(contour) - points)
    except ZeroDivisionError as e:
        print ('len(contour) = {} smaller, points = {}'.format(
            len(contour), points))
        raise ValueError('len(contour) = {} smaller, points = {}'.format(
            len(contour), points)) from e
        
    short_contour = np.delete(contour, np.arange(0, contour.size, step), axis=0)
    
    return short_contour

In [19]:
def get_contour_from_mask(mask_name, kernel_size, points):
    '''Returns the image, the longest contour, and shortened contour 
       given a mask name and fixed number of points.'''
    
    image = cv2.imread(mask_name)
    contour = extract_longest_contour(image, kernel_size)
    try:
        short_contour = shorten_contour(contour, points)
    except ValueError as e:
        print(mask_name)
        raise ValueError(mask_name) from e
        
    return image, contour, short_contour

## Create contours for all masks

In [25]:
def create_contours_for_all_masks(annotations_folder_path, contours_folder_path):
    
    annotations_folders = os.listdir(annotations_folder_path)
    
    # Iterate through folders
    for i, folder in enumerate(annotations_folders):
        
        if (i > 0):
            break
            
        print('Folder {}'.format(folder))
        
        # Create folder to save contours
        contours_folder = os.path.join(contours_folder_path, folder)
        if not os.path.exists(contours_folder):
            os.makedirs(contours_folder)
        
        print('\tCreating contours')
        
        annotations = os.listdir(os.path.join(annotations_folders, folder))
        
        # Iterate through images
        for j, annotation in enumerate(annotations):

            if (j > 0):
                break

            # If first mask, then extract 256 contour points
            if re.match(r"00000.png", mask_name):
                try:
                    image, contour, short_contour = get_contour_from_mask(
                        mask_name, kernel_size, 256)
                except ValueError as e:
                    continue
            # If not, then 512 contour points
            else:
                try:
                    image, contour, short_contour = get_contour_from_mask(
                        mask_name, kernel_size, 512)
                except ValueError as e:
                    continue

            # Save contour
            save_image_with_contour(image, short_contour, os.path.join(contours_folder, annotation))

In [26]:
create_contours_for_all_masks("DAVIS_2016/DAVIS/Annotations/480p/", "DAVIS_2016/DAVIS/Contours/480p/")

Folder boat


PermissionError: [Errno 13] Permission denied: 'DAVIS_2016/DAVIS/Contours'